# Find the minimal medium #

In [1]:
import pycomo
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cobra

2025-11-17 16:18:20,157 - pycomo - INFO - Logger initialized.
2025-11-17 16:18:20,160 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 16:18:20,161 - pycomo - INFO - Utils Logger initialized.
2025-11-17 16:18:20,163 - pycomo - INFO - Multiprocess Logger initialized.


In [2]:
pycomo.configure_logger(level="warning")

## Create the community model ##

First, we load the single organism models of the members. The models were automatically reconstructed using gapseq (see [Zimmermann et al. 2021](https://doi.org/10.1186/s13059-021-02295-1)), based on genome sequencing data:
 - M. smithii: [ABYW00000000.1](https://www.ncbi.nlm.nih.gov/nuccore/ABYW00000000.1/)
 - B. thetaiotaomicron [NC_004663.1](https://www.ncbi.nlm.nih.gov/nuccore/NC_004663.1)

A full description of the model reconstruction can be found in [Duller et al. 2024](https://doi.org/10.1038/s41467-024-52037-7)

In [3]:
single_models = "data/member_models"
named_models = pycomo.load_named_models_from_dir(single_models)

In [4]:
named_models

{'ABYW00000000.1_ms_medium_gapseq_open': <Model ABYW00000000_1 at 0x7f7d90b8a150>,
 'NC_004663.1_ms_b_tim_medium_gapseq_open': <Model NC_004663_1 at 0x7f7d9020df50>}

In [5]:
prepared_models = [pycomo.SingleOrganismModel(m, n) for n, m in named_models.items()]
com_model = pycomo.CommunityModel(prepared_models, name="com")

2025-11-17 16:18:27,103 - pycomo - WARNING - Warning: model name ABYW00000000.1_ms_medium_gapseq_open is not compliant with sbml id standards and was changed to ABYW00000000_1_ms_medium_gapseq_open
2025-11-17 16:18:27,338 - pycomo - WARNING - Warning: model name NC_004663.1_ms_b_tim_medium_gapseq_open is not compliant with sbml id standards and was changed to NC_004663_1_ms_b_tim_medium_gapseq_open


A report of the community model

In [6]:
print(f"Reactions: {len(com_model.model.reactions)}")
print(f"Metabolites: {len(com_model.model.metabolites)}")

2025-11-17 16:19:00,003 - pycomo - WARNING - Not all reactions in the model are mass and charge balanced. To check which reactions are imbalanced, please run the get_unbalanced_reactions method of this CommunityModel object


Reactions: 9290
Metabolites: 8786


## Load and apply medium ##

In [7]:
medium_dict = pycomo.read_medium_from_file("data/media/ms_b_tim_medium_gapseq_open.csv", "_medium")
for k, v in medium_dict.items():
    medium_dict[k] = 1000.
com_model.medium = medium_dict
com_model.apply_medium()

Name,com
Memory address,7f7d8cd313d0
Number of metabolites,8786
Number of reactions,9290
Number of genes,1141
Number of groups,987
Objective expression,1.0*community_biomass - 1.0*community_biomass_reverse_44dc1
Compartments,"ABYW00000000_1_ms_medium_gapseq_open_c0, ABYW00000000_1_ms_medium_gapseq_open_e0, ABYW00000000_1_ms_medium_gapseq_open_p0, ABYW00000000_1_ms_medium_gapseq_open_bio, ABYW00000000_1_ms_medium_gapseq_open_medium, medium, fraction_reaction, NC_004663_1_ms_b_tim_medium_gapseq_open_c0, NC_004663_1_ms_b_tim_medium_gapseq_open_e0, NC_004663_1_ms_b_tim_medium_gapseq_open_p0, NC_004663_1_ms_b_tim_medium_gapseq_open_bio, NC_004663_1_ms_b_tim_medium_gapseq_open_medium"


## Save the model ##

In [8]:
com_model.save("data/community_models/M_smithii_B_tim_ms_b_tim_medium_gapseq_open_unconstr.xml")

In [9]:
com_model = pycomo.CommunityModel.load("data/community_models/M_smithii_B_tim_ms_b_tim_medium_gapseq_open_unconstr.xml")

## Set the transport reaction bounds to physiologically feasible range ##
Since data for transport fluxes is not present, we set transport bounds to $\pm 10 mmol * gDW^{-1} * h^{-1}$

def is_transporter(rxn, member_names):
    if "_TF_" in rxn.id or rxn.id[-3:] in ["_lb", "_ub"]:
        return False
    met_comps = [met.compartment for met in rxn.metabolites.keys() if met.compartment != "fraction_reaction"]
    if len(set(met_comps)) < 2:
        return False
    for n in member_names:
        if n in rxn.id and all([n in m for m in met_comps]):
            return True
    return False

In [10]:
member_names = com_model.get_member_names()
name_conv = {"A": "M. smithii", "N": "B. thetaiotaomicron"}

member_abbrev = [name_conv[n[0]] for n in member_names]

In [11]:
tp_val = 10.
for tp_rxn in com_model.model.reactions.query(lambda r: com_model.is_transporter(r)):
    lb = -tp_val if tp_rxn.lower_bound < -tp_val else tp_rxn.lower_bound
    ub = tp_val if tp_rxn.upper_bound > tp_val else tp_rxn.upper_bound
    com_model.change_reaction_bounds(tp_rxn, lower_bound=lb, upper_bound=ub)

## Find a minimal medium for the community, based on the MS medium ##

In [12]:
com_model.convert_to_fixed_abundance()
min_med = cobra.medium.minimal_medium(com_model.model)
print(min_med)

EX_cpd00009_medium    0.098943
EX_cpd00030_medium    0.000365
EX_cpd00034_medium    0.000365
EX_cpd00051_medium    0.134785
EX_cpd00119_medium    0.210719
EX_cpd00149_medium    0.000585
EX_cpd00244_medium    0.000094
EX_cpd10516_medium    0.000365
EX_cpd00084_medium    0.007729
EX_cpd00063_medium    0.000365
EX_cpd00058_medium    0.000365
EX_cpd10515_medium    0.000759
EX_cpd00060_medium    0.017689
EX_cpd00205_medium    0.000365
EX_cpd00254_medium    0.000365
EX_cpd00393_medium    0.000597
EX_cpd00220_medium    0.000395
EX_cpd00048_medium    0.001331
EX_cpd00099_medium    0.000365
EX_cpd90020_medium    0.002532
dtype: float64


In [13]:
medium_dict = min_med.to_dict()

medium_df_rows = []

for exchg_rxn in medium_dict.keys():
    met = list(com_model.model.reactions.get_by_id(exchg_rxn).metabolites.keys())[0]
    met_id = met.id.replace("_medium","")
    met_name = met.name.replace("_medium", "")
    medium_df_rows.append({"compounds": met_id, "name": met_name, "maxFlux": 1000})

medium_df = pd.DataFrame.from_dict(medium_df_rows)

medium_df.to_csv("data/media/minimal_medium_community.csv", index=False, sep=",")

medium_df

,compounds,name,maxFlux
0,cpd00009,Phosphate,1000
1,cpd00030,Mn2+,1000
2,cpd00034,Zn2+,1000
3,cpd00051,L-Arginine,1000
4,cpd00119,L-Histidine,1000
5,cpd00149,Co2+,1000
6,cpd00244,Ni2+,1000
7,cpd10516,fe3,1000
8,cpd00084,L-Cysteine,1000
9,cpd00063,Ca2+,1000


## Apply the minimal medium to the model ##

In [14]:
medium_dict = min_med.to_dict()
for k, v in medium_dict.items():
    medium_dict[k] = 1000.
com_model.medium = medium_dict
com_model.apply_medium()


Name,com
Memory address,7f7d85a4dad0
Number of metabolites,9008
Number of reactions,9512
Number of genes,1141
Number of groups,987
Objective expression,1.0*community_biomass - 1.0*community_biomass_reverse_44dc1
Compartments,"ABYW00000000_1_ms_medium_gapseq_open_c0, ABYW00000000_1_ms_medium_gapseq_open_e0, ABYW00000000_1_ms_medium_gapseq_open_p0, ABYW00000000_1_ms_medium_gapseq_open_bio, ABYW00000000_1_ms_medium_gapseq_open_medium, medium, fraction_reaction, NC_004663_1_ms_b_tim_medium_gapseq_open_c0, NC_004663_1_ms_b_tim_medium_gapseq_open_e0, NC_004663_1_ms_b_tim_medium_gapseq_open_p0, NC_004663_1_ms_b_tim_medium_gapseq_open_bio, NC_004663_1_ms_b_tim_medium_gapseq_open_medium"


## Save the model ##

In [15]:
com_model.save(f"data/community_models/{member_abbrev[0]}_{member_abbrev[1]}_ms_open_min_med_tp_10.xml")

## Find the maximum growth rate ##

_M. smithii_ growth rate

In [16]:
member_names

['ABYW00000000_1_ms_medium_gapseq_open',
 'NC_004663_1_ms_b_tim_medium_gapseq_open']

In [17]:
com_model.convert_to_fixed_abundance()
com_model.apply_fixed_abundance({member_names[0]: 1., member_names[1]: 0.})
solution = com_model.model.optimize()
print(f"Growth rate of M. smithii: {solution.objective_value} (solver status: {solution.status})")

Growth rate of M. smithii: 0.0 (solver status: optimal)


_B. thetaiotaomicron_ growth rate

In [18]:
com_model.convert_to_fixed_abundance()
com_model.apply_fixed_abundance({member_names[0]: 0., member_names[1]: 1.})
solution = com_model.model.optimize()
print(f"Growth rate of B. thetaiotaomicron: {solution.objective_value} (solver status: {solution.status})")


Growth rate of B. thetaiotaomicron: 0.7754058387708818 (solver status: optimal)


Community growth rate

In [19]:
com_model.max_growth_rate(sensitivity=5, return_abundances=True)

2025-11-17 16:22:56,003 - pycomo - INFO - Logger initialized.
2025-11-17 16:22:56,005 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 16:22:56,005 - pycomo - INFO - Utils Logger initialized.
2025-11-17 16:22:56,006 - pycomo - INFO - Multiprocess Logger initialized.
2025-11-17 16:22:57,940 - pycomo - INFO - Logger initialized.
2025-11-17 16:22:57,941 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 16:22:57,941 - pycomo - INFO - Utils Logger initialized.
2025-11-17 16:22:57,941 - pycomo - INFO - Multiprocess Logger initialized.
2025-11-17 16:23:24,624 - pycomo - INFO - Logger initialized.
2025-11-17 16:23:24,624 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 16:23:24,624 - pycomo - INFO - Utils Logger initialized.
2025-11-17 16:23:24,625 - pycomo - INFO - Multiprocess Logger initialized.
2025-11-17 16:23:26,708 - pycomo - INFO - Logger initialized.
2025-11-17 16:23:26,708 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 16:23:26,709

,reaction_id,min_flux,max_flux
0,ABYW00000000_1_ms_medium_gapseq_open_fraction_...,0.478045,0.479571
1,NC_004663_1_ms_b_tim_medium_gapseq_open_fracti...,0.520429,0.521955
2,community_biomass,0.851180,0.851180
